In [ ]:
import pandas as pd
pd.set_option('precision', 3)

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import os

import mxnet as mx
from gluonts.model.n_beats import NBEATSEnsembleEstimator
from gluonts.mx import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions

In [ ]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]
output_path = "./output/nbeats_TEST"
if not os.path.exists(output_path):
    if not os.path.exists("./output"):
        os.mkdir("./output")
    os.mkdir(output_path)

## Load Data From File

In [ ]:
foodprice_categories = pd.read_csv("./foodprice_categories.txt", sep='\n', header=None)[0].to_list()
foodprice_df = pd.read_csv("./all_data.csv", index_col=0)
foodprice_df = foodprice_df.set_index(pd.DatetimeIndex(foodprice_df.index))
foodprice_df

## Experiment And Data Setup

In [ ]:
report_sim_dates = pd.read_csv("./reportsimdates.txt", sep='\n', header=None)[0].to_list()
report_sim_dates

In [ ]:
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

In [ ]:
dataset_df = foodprice_df[foodprice_categories].T
dataset_df

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}
def get_prophet_df(foodprice_df, food_category, dates):
    df = foodprice_df[food_category][dates]
    df = df.reset_index()
    df = df.rename({'REF_DATE':'ds', 'index':'ds', food_category:'y'}, axis=1)
    return df

## Fit Models and Save Forecasts

Fit a model using data up to the `report_sim_date` and use it to produce forecasts for the following 18 months. Collect evaluation metrics for comparion and save the forecasts for use in the ensembling experiments.

In [ ]:
def train_eval_nbeats(report_sim_date, prediction_length=18):

    report_train_dates = sim_train_dates[report_sim_date]
    report_valid_dates = sim_valid_dates[report_sim_date]
    
    # train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
    train_ds = ListDataset(
        [{'target': x, 'start': report_sim_date} for x in dataset_df[list(report_train_dates)].values],
        freq='MS'
    )

    valid_ds_report = ListDataset(
        [{'target': x, 'start': report_sim_date} for x in dataset_df[list(report_train_dates) + list(report_valid_dates)].values],
        freq='MS'
    )

    estimator = NBEATSEnsembleEstimator(
        prediction_length=prediction_length,
        meta_bagging_size = 3,
        meta_context_length = [prediction_length * m for m in [3,5,7]], 
        meta_loss_function = ['sMAPE'], 
        num_stacks = 30,
        widths= [512],
        freq="MS",
        trainer=Trainer(
                    epochs=1,  # 50
                    ctx=mx.context.cpu()
                )

    )

    predictor = estimator.train(train_ds)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=valid_ds_report,  # test dataset
        predictor=predictor,  # predictor
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)
    all_fc_dates = list(report_train_dates) + list(report_valid_dates)

    all_food_metrics = {}
    food_forecasts = {}

    for target_index in range(len(forecasts)):

        # Get food price category
        foodprice_category = foodprice_df.columns[target_index]

        # plot actual
        fig, ax = plt.subplots(figsize=(8,3))
        ax.scatter(all_fc_dates, foodprice_df[foodprice_category][all_fc_dates], color='black')

        # plot forecast
        forecast_entry = forecasts[target_index]
        ax.plot(report_valid_dates, forecast_entry.mean[:len(report_valid_dates)], color='C0')

        plt.title(f"{foodprice_category}, {report_sim_date}")
        plt.grid()
        plt.show()

        fc_metrics = {}
        for metric_name, metric_fn in metrics.items():
            print(metric_name, metric_fn)
            y_true = foodprice_df[foodprice_category][report_valid_dates]
            y_pred = forecast_entry.mean[:len(report_valid_dates)]
            score = metric_fn(y_true=y_true, y_pred=y_pred)
            print(metric_name, score)
            fc_metrics[metric_name] = score


        all_food_metrics[foodprice_category] = fc_metrics
        food_forecasts[foodprice_category] = pd.Series(forecast_entry.mean[:len(report_valid_dates)], index=report_valid_dates, name=foodprice_category)


    all_forecasts = pd.DataFrame(food_forecasts)
    all_forecasts.to_csv(f"{output_path}/forecasts_{report_sim_date}.csv")

    return all_food_metrics, all_forecasts

In [ ]:
all_valid_metrics = {}
all_forecasts = {}

for report_sim_date in report_sim_dates:
    valid_metrics, forecasts = train_eval_nbeats(report_sim_date)
    all_valid_metrics[report_sim_date] = valid_metrics
    all_forecasts[report_sim_date] = forecasts

In [ ]:
valid_metrics_concat = {}

all_valid_metrics.keys()

for report_date, valid_scores in all_valid_metrics.items():
    valid_metrics_concat[report_date] = pd.DataFrame(valid_scores).T
index = valid_metrics_concat[report_date].index
columns = valid_metrics_concat[report_date].columns
scores = [df.values for date, df in valid_metrics_concat.items()]
mean_scores = pd.DataFrame(np.array(scores).mean(axis=0), index=index, columns=columns)
mean_scores.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
mean_scores